In [1]:
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../dashgrid'))
sys.path.append(os.path.abspath('../../dashgrid/dashgrid'))
from dashgrid import dgrid
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
import dash_table
import pandas as pd
import numpy as np
from dashgrid import dgrid


In [2]:
def create_dt_div(dtable_id,df_in=None,
                  columns_to_display=None,
                  editable_columns_in=None,
                  title='Dash Table'):
    editable_columns = [] if editable_columns_in is None else editable_columns_in
    datatable_id = dtable_id
    dt = dash_table.DataTable(
        page_current= 0,
        page_size= 100,
        page_action='native',
        sort_action='native',
        filter_action='none', # 'fe',
#             content_style='grow',
        style_cell_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ] + [
            {
                'if': {'column_id': c},
                'textAlign': 'left',
            } for c in ['symbol', 'underlying']
        ],

        style_as_list_view=False,
        style_table={
            'maxHeight':'450','overflowX': 'scroll',
        } ,
        editable=True,
        css=[{"selector": "table", "rule": "width: 100%;"}],
        id=datatable_id
    )
    if df_in is None:
        df = pd.DataFrame({'no_data':[]})
    else:
        df = df_in.copy()
        if columns_to_display is not None:
            df = df[columns_to_display]                
    dt.data=df.to_dict("rows")
    dt.columns=[{"name": i, "id": i,'editable': True if i in editable_columns else False} for i in df.columns.values]                    
    print(f'dt.columns {dt.columns}')
    child_div = html.Div([html.H4(title,style={'height':'3px'}),dt])
    return child_div

def file_store_transformer(contents):
    if contents is None or len(contents)<=0 or contents[0] is None:
        d =  None
    else:
        d = dgrid.parse_contents(contents).to_dict('rows')
    return d


In [3]:
button_style={
    'line-height': '40px',
    'borderWidth': '1px',
    'borderStyle': 'dashed',
    'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}
u1 = dcc.Upload(
            id='u1',
            children=html.Div(['enter a csv file']),
            # Allow multiple files to be uploaded
            multiple=False,
            style=button_style)

df_init = pd.DataFrame({'symbol':['spy','ibm'],'position':[100,-100]})
cols=['symbol','position']
s1 = dcc.Store(id='s1')#,data=df_init.to_dict('rows'))
s2 = dcc.Store(id='s2')

dtable_div = create_dt_div('dtable',df_in=df_init,
                           columns_to_display=cols,
                           editable_columns_in=['position'])
dt1 = html.Div([dtable_div],id='dt1')
gr = dgrid.GridGraph('g1', 'graph 1',(dt1.id,'data'),
                    df_x_column='symbol',df_y_columns=['position'],plot_bars=True)
gr1 = gr.html
gr1.id = 'gr1'

u1_lambda = lambda value_list: [None,None] if value_list[0] is None else [file_store_transformer(value_list[0]),None]
u1_comp = dgrid.ComponentWrapper(u1,
                 input__tuples=[(u1.id,'contents')],
                 output_tuples=[(s1.id,'data'),(s2.id,'data')],
                 callback_input_transformer=u1_lambda)

# dt1_lambda = lambda value_list: [None] if value_list[0] is None else [create_dt_div('dtable',df_in=pd.DataFrame(value_list[0]),editable_columns_in=['position]'])]
def dt1_lambda(value_list): 
    print(f'dt1_lambda value_list: {value_list}')
    ret = [None,None]
    try:
        if value_list[0] is not None:
            dict_df = value_list[0]
            df = pd.DataFrame(dict_df)
            dt_div = create_dt_div('dtable',df_in=df,
                        columns_to_display=cols,
                        editable_columns_in=['position'])
            ret =  [dt_div,dict_df]
    except Exception as e:
        print(str(e))
    print(f'dt1_lambda ret: {ret}')
    return ret

dt1_comp = dgrid.ComponentWrapper(dt1,
                 input__tuples=[(s1.id,'data')],
                 output_tuples=[(dt1.id,'children'),('dtable','data')],
                 callback_input_transformer=dt1_lambda)


def gr1_lambda(value_list): 
    print(f'gr1_lambda value_list: {value_list}')
    ret = [None]
    try:
        if value_list[0] is not None:
            df = pd.DataFrame(value_list[0])
            ret =  [gr.make_chart_figure(df)]
    except Exception as e:
        print(str(e))
    print(f'gr1_lambda ret: {ret}')
    return ret

gr1_comp = dgrid.ComponentWrapper(gr1,
                 input__tuples=[('dtable','data')],
                 output_tuples=[(gr1.children.id,'figure')],
                 callback_input_transformer=gr1_lambda)

app = dash.Dash()
app.layout = html.Div([u1_comp.html,s1,s2,dt1_comp.html,gr1_comp.html])
# [c.callback(app) for c in [u1_comp,dt1_comp]]
u1_comp.callback(app)
dt1_comp.callback(app)
gr1_comp.callback(app)



dt.columns [{'name': 'symbol', 'id': 'symbol', 'editable': False}, {'name': 'position', 'id': 'position', 'editable': True}]


<function dashgrid.dgrid.ComponentWrapper.callback.<locals>.execute_callback(*inputs_and_states)>

In [ ]:
app.run_server(host='127.0.0.1',port=8500)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2019-07-18 01:13:34,227 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2019-07-18 01:13:39,400 - werkzeug - INFO - 127.0.0.1 - - [18/Jul/2019 01:13:39] "GET / HTTP/1.1" 200 -
2019-07-18 01:13:39,477 - werkzeug - INFO - 127.0.0.1 - - [18/Jul/2019 01:13:39] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-18 01:13:39,478 - werkzeug - INFO - 127.0.0.1 - - [18/Jul/2019 01:13:39] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-18 01:13:39,484 - werkzeug - INFO - 127.0.0.1 - - [18/Jul/2019 01:13:39] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-18 01:13:39,494 - werkzeug - INFO - 127.0.0.1 - - [18/Jul/2019 01:13:39] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1563025508 HTTP/1.1" 200 -
2019-07-18 01:13:39,504 - werkzeug - INFO - 

gr1_lambda value_list: [[{'symbol': 'spy', 'position': 100}, {'symbol': 'ibm', 'position': -100}]]
dt1_lambda value_list: [None]
dt1_lambda ret: [None, None]
gr1_lambda ret: [Figure({
    'data': [{'type': 'bar',
              'uid': 'cdff665e-a91a-11e9-a879-8c859025e7f9',
              'x': array(['spy', 'ibm'], dtype=object),
              'y': array([ 100, -100])}],
    'layout': {'plot_bgcolor': '#f5f5f0', 'title': 'graph 1'}
})]


2019-07-18 01:13:40,371 - werkzeug - INFO - 127.0.0.1 - - [18/Jul/2019 01:13:40] "POST /_dash-update-component HTTP/1.1" 200 -


dt1_lambda value_list: [None]
dt1_lambda ret: [None, None]


2019-07-18 01:13:47,285 - werkzeug - INFO - 127.0.0.1 - - [18/Jul/2019 01:13:47] "POST /_dash-update-component HTTP/1.1" 200 -
2019-07-18 01:13:47,315 - werkzeug - INFO - 127.0.0.1 - - [18/Jul/2019 01:13:47] "POST /_dash-update-component HTTP/1.1" 200 -
2019-07-18 01:13:47,445 - werkzeug - INFO - 127.0.0.1 - - [18/Jul/2019 01:13:47] "POST /_dash-update-component HTTP/1.1" 200 -
2019-07-18 01:13:47,480 - werkzeug - INFO - 127.0.0.1 - - [18/Jul/2019 01:13:47] "POST /_dash-update-component HTTP/1.1" 200 -


dt1_lambda value_list: [[{'symbol': 'ZSY00', 'position': -50}, {'symbol': '^EURUSD', 'position': 200000}, {'symbol': 'XLU', 'position': -200}, {'symbol': 'XLE', 'position': 200}, {'symbol': 'USO', 'position': 2}, {'symbol': 'SPY', 'position': -200}, {'symbol': 'SOYB', 'position': 3500}, {'symbol': 'ZCY00', 'position': -50}, {'symbol': 'CLY00', 'position': 500}, {'symbol': 'HOY00', 'position': 21000}, {'symbol': 'HGU19', 'position': 25000}, {'symbol': 'CPER', 'position': -2600}, {'symbol': 'CTZ19', 'position': 500}]]
dt.columns [{'name': 'symbol', 'id': 'symbol', 'editable': False}, {'name': 'position', 'id': 'position', 'editable': True}]
dt1_lambda ret: [Div([H4(children='Dash Table', style={'height': '3px'}), DataTable(columns=[{'name': 'symbol', 'id': 'symbol', 'editable': False}, {'name': 'position', 'id': 'position', 'editable': True}], css=[{'selector': 'table', 'rule': 'width: 100%;'}], data=[{'symbol': 'ZSY00', 'position': -50}, {'symbol': '^EURUSD', 'position': 200000}, {'symb

In [ ]:
# dtable_div.children[1].id

In [ ]:
dtable_div.children[1]